In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园-使用MediaPipe生成图像

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_mediapipe_image_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
  </td>
</table>

**注意**：本笔记本已在以下环境中进行测试：

- Python版本= 3.9

**注意**：此Colab中链接的检查点和数据集不是由谷歌拥有或分发的，而是由第三方提供的。在使用检查点和数据之前，请先查看第三方提供的条款和条件。

## 概述

本笔记本演示了如何通过添加低秩适应（LoRA）权重来自定义[MediaPipe图像生成器](https://developers.google.com/mediapipe/solutions/vision/image_generator)，即文本到图像生成器，从而生成特定人物、物体和风格的图像。

使用Vertex AI的Model Garden，我们将在特定概念的专用数据集上对标准扩散模型进行重新训练，这些概念由唯一标记识别。经过训练后带有新的LoRA权重，新模型能够在文本提示中指定标记时生成新概念的图像。

一旦使用LoRA权重定制了模型，应仅用于生成标记概念的图像。它不再适用于通用图像生成模型。有关使用LoRA权重定制MediaPipe图像生成器的更多信息，请参阅[MediaPipe文档](https://developers.google.com/mediapipe/solutions/vision/image_generator#lora)。

注意：如果您正在创建LoRA权重以生成特定人物和面部的图像，请仅在您自己的脸部或已经允许您这样做的人的面部上使用此解决方案。

### 目标

* 在 Google Cloud 项目中设置 Vertex AI。
* 在专门的数据集上训练文本到图像扩散模型，以创建 [LoRA](https://arxiv.org/abs/2106.09685) 权重。
* 将通用图像生成器定制为专门的生成器，可以向生成的图像中注入特定的对象、人物和风格。
* 配置新训练的图像生成器。
* 下载、上传和部署新模型。

### 成本

此教程使用 Google Cloud 的付费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

在你开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager).

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).
4. 如果您是在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk).

5. 在下面的单元格中输入您的项目ID。然后运行单元格，确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

### 在Colab上进行身份验证
**注意**: 如果您没有使用[Colab](https://colab.google/),请跳过这一步。

运行以下命令在Colab上安装依赖项并完成与Google Cloud的身份验证。

In [ ]:
! pip3 install --upgrade pip

import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 设置您的项目 ID (`PROJECT_ID`)

如果您不知道您的项目 ID，可以尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 设置存储位置（`REGION`）

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于[Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)的信息。

In [ ]:
REGION = ""  # @param {type: "string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集和训练模型。

In [ ]:
BUCKET_URI = ""  # @param {type:"string"}

如果您的存储桶尚不存在，请创建您的云存储存储桶。

**注意**：如果您还没有存储桶，请运行以下单元。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import json
import os
from datetime import datetime

from google.cloud import aiplatform

### 初始化用于Python的Vertex AI SDK

为您的项目初始化用于Python的Vertex AI SDK。

In [ ]:
now = datetime.now().strftime("%Y%m%d-%H%M%S")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temp/%s" % now)

MODEL_EXPORT_PATH = os.path.join(STAGING_BUCKET, "model")

IMAGE_EXPORT_PATH = os.path.join(STAGING_BUCKET, "image")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

定义培训和服务的常量

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "mediapipe_stable_diffusion_%s" % now
TRAINING_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/mediapipe-stable-diffusion-train"
TRAINING_MACHINE_TYPE = "a2-highgpu-1g"
TRAINING_ACCELERATOR_TYPE = "NVIDIA_TESLA_A100"
TRAINING_ACCELERATOR_COUNT = 1

PREDICTION_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-serve"
PREDICTION_PORT = 7080
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
PREDICTION_MACHINE_TYPE = "n1-standard-8"
UPLOAD_MODEL_NAME = "mediapipe_stable_diffusion_model_%s" % now

## 训练一个定制的图像生成器

在这一部分，我们将通过在[DreamBooth数据集](https://github.com/google/dreambooth/tree/main)中的[茶壶](https://github.com/google/dreambooth/tree/main/dataset/teapot)图像上训练模型来定制图像生成器。使用通过训练创建的LoRA权重，新模型将能够将茶壶注入生成的图像中。

这是一个简单的示例实现。您可以修改以下单元格以进一步定制笔记本。

### 选择预训练模型进行下载

MediaPipe图像生成器任务要求您下载一个符合`runwayml/stable-diffusion-v1-5 EMA-only`模型格式的训练模型，基于以下模型：[runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/)。

In [ ]:
unet_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/unet/diffusion_pytorch_model.bin"  # @param {type:"string"}
vae_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/vae/diffusion_pytorch_model.bin"  # @param {type:"string"}
text_encoder_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/text_encoder/pytorch_model.bin"  # @param {type:"string"}

### 为训练准备输入数据

为图像生成定制模型需要一个包含想要在生成中使用的概念实例的示例图片的数据集。这个概念可以是一个人、物体或风格。

**物体**
![](https://storage.googleapis.com/mediapipe-assets/documentation/object_lora.png)

**人物**
![](https://storage.googleapis.com/mediapipe-assets/documentation/person_lora.png)

**风格**
![](https://storage.googleapis.com/mediapipe-assets/documentation/style_lora.png)

您还必须为新概念分配一个唯一的标记。提示应该包含该标记，即在本例中是"monadikos"，然后是描述要生成的概念的词汇。在这个例子中，我们使用"A monadikos teapot"。可以从Google Cloud Storage下载[茶壶](https://github.com/google/dreambooth/tree/main/dataset/teapot)数据集中的图像。

定制模型将识别术语"monadikos teapot"，并将一个茶壶的图像注入到生成的图像中。

In [ ]:
# Path to the training data folder.
training_data_path = "gs://mediapipe-tasks/image_generator/teapot"  # @param {type:"string"}
# An instance description of the training data.
training_data_prompt = "A monadikos teapot"  # @param {type:"string"}

### 设置训练选项

图像生成器配备了一组预定义的超参数（`HParams`）设置，适用于特定情况下效果最佳。您应选择与您的用例最匹配的模板。

您可以进一步定制超参数，比如学习率和训练步数（epochs）。有关这些超参数的更多信息，请参阅[Google机器学习词汇表](https://developers.google.com/machine-learning/glossary)。

要设置自定义训练参数，请调整以下超参数的值：

In [ ]:
# Parameters about training configuration
# The learning rate to use for gradient descent training.
learning_rate: float = 0.00001  # @param {type:"number"}
# Number of training steps. If set to 0, uses the default value.
num_train_steps: int = 0  # @param {type:"integer"}
# Save the checkpoint in every n steps.
save_checkpoints_every_n: int = 100  # @param {type:"integer"}
# Batch size for training.
batch_size: int = 1  # @param {type:"integer"}

# Dataset-related parameters
# Whether to use random horizontal flip on data.
random_flip: bool = False  # @param {type:"boolean"}
# Whether to use random largest square crop.
random_crop: bool = False  # @param {type:"boolean"}
# Whether to distort the color of the image (jittering order is random).
random_color_jitter: bool = False  # @param {type:"boolean"}

# Hyperparameters for LoRA tuning
# The rank in the low-rank matrices. If set to 0, uses the default value.
lora_rank: int = 0  # @param {type:"integer"}

另外，您也可以使用我们预先训练好的模型之一来应用这些模板。这些模板已经定制好，并且已经包含LoRA权重：
* [物体（浆果碗）](https://storage.googleapis.com/mediapipe-tasks/image_generator/object/pytorch_lora_weights.bin)
* [人脸](https://storage.googleapis.com/mediapipe-tasks/image_generator/face/pytorch_lora_weights.bin)
* [风格](https://storage.googleapis.com/mediapipe-tasks/image_generator/style/pytorch_lora_weights.bin)

In [ ]:
template = ""  # @param ["", "face", "object", "style"]

测试自定义的图像生成器模型

在训练完自定义模型之后，我们将生成图像来检查自定义模型的质量。您可以在下方提供文本提示并配置生成测试图像的选项。

### 定义测试生成提示

指定要用于测试定制模型的提示。请注意，提示中包含 "monadikos teapots" 的变体。如果您正在使用另一个数据集自定义此笔记本，则请设置一个代表训练数据中所描绘的对象、人物或风格的标记。

In [ ]:
prompt: str = "Two monadikos teapots on a table"  # @param {type:"string"}

### 配置参数以生成测试图像

设置配置选项以使用定制模型运行图像生成。

In [ ]:
# Number of steps to run inference.
number_inference_steps: int = 50  # @param {type:"integer"}
# Classifier-free guidance weight to use during inference. Weight must be is >= 1.0.
guidance_scale: float = 7.5  # @param {type:"number"}
#  Number of generated images per prompt.
number_generated_images: int = 8  # @param {type:"integer"}

### 使用LoRA调优图像生成器
使用LoRA调优图像生成器，并根据您的提示生成新的图像。 在具有A100 GPU的Vertex AI上，这可能需要最多10分钟。

In [ ]:
model_export_path = MODEL_EXPORT_PATH
image_export_path = IMAGE_EXPORT_PATH

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAINING_MACHINE_TYPE,
            "accelerator_type": TRAINING_ACCELERATOR_TYPE,
            "accelerator_count": TRAINING_ACCELERATOR_COUNT,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER,
            "command": [],
            "args": [
                "--task_name=stable_diffusion",
                "--model_export_path=%s" % model_export_path,
                "--image_export_path=%s" % image_export_path,
                "--training_data_path=%s" % training_data_path,
                "--training_data_prompt='%s'" % training_data_prompt,
                "--prompt='%s'" % prompt,
                "--hparams_template=%s" % template,
                "--hparams=%s"
                % json.dumps(
                    {
                        "learning_rate": learning_rate,
                        "num_train_steps": num_train_steps,
                        "save_checkpoints_every_n": save_checkpoints_every_n,
                        "batch_size": batch_size,
                        "random_flip": random_flip,
                        "random_crop": random_crop,
                        "random_color_jitter": random_color_jitter,
                        "lora_rank": lora_rank,
                        "torch_vae": vae_url,
                        "torch_unet": unet_url,
                        "torch_text_encoder": text_encoder_url,
                    }
                ),
                "--generator_hparams=%s"
                % json.dumps(
                    {
                        "number_inference_steps": number_inference_steps,
                        "guidance_scale": guidance_scale,
                        "number_generated_images": number_generated_images,
                    }
                ),
            ],
        },
    }
]

training_job = aiplatform.CustomJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

training_job.run()

## 下载图片和模型

在训练和测试新模型之后，您可以下载生成的图片和新定制的模型。训练中的 LoRA 权重也可用于 MediaPipe Tasks ImageGenerator API 用于设备上的应用程序。

### 下载生成的图像

在不同的检查点上下载和预览生成的图像。
检查生成的图像有助于确定最佳的检查点，并避免欠拟合或过拟合。

In [ ]:
import sys

import matplotlib.pyplot as plt


def copy_image(images_source, images_dest):
    os.makedirs(images_dest, exist_ok=True)
    ! gsutil cp -r {images_source}/* {images_dest}


local_image_path = "./images/"
copy_image(IMAGE_EXPORT_PATH, local_image_path)

steps_samples = {}
for filename in os.listdir(local_image_path):
    absolute_path = os.path.join(local_image_path, filename)
    if os.path.isfile(absolute_path):
        parsed_name = filename.split("_")
        step = int(parsed_name[1])
        if step not in steps_samples:
            steps_samples[step] = []
        image = plt.imread(absolute_path)
        steps_samples[step].append(image)

for step in sorted(steps_samples.keys()):
    print(f"\nGenerated image with training steps {step}:")
    for image in steps_samples[step]:
        plt.figure(figsize=(20, 10), dpi=150)
        plt.axis("off")
        plt.imshow(image)
        plt.show()

默认情况下，最后一个检查点用于部署。然而，根据以上视觉检查，我们可以在这里进行定制。

In [ ]:
deployed_checkpoint: int = -1  # @param {type:"integer"}
if deployed_checkpoint == -1:
    deployed_checkpoint = num_train_steps
valid_checkpoints = list(
    range(save_checkpoints_every_n, num_train_steps + 1, save_checkpoints_every_n)
)
if deployed_checkpoint not in valid_checkpoints:
    raise ValueError("Invalid checkpoint chosen for deployment.")

下载模型

在微调和评估模型之后，您可以下载模型和检查点。

In [ ]:
import sys


def copy_model(model_source, model_dest):
    os.makedirs(model_dest, exist_ok=True)
    ! gsutil -m cp -r {model_source}/* {model_dest}


local_model_path = "/models"
copy_model(MODEL_EXPORT_PATH, local_model_path)

! tar czf models.tar.gz {local_model_path}/*

if "google.colab" in sys.modules:
    from google.colab import files

    files.download("models.tar.gz")

将以下英文文本翻译为中文：# 上传并部署到Vertex AI

本部分展示了如何使用训练好的模型进行测试。
1. 将模型上传并部署到[Vertex AI模型注册](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)
2. 从部署的模型获取[在线预测](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions)。

### 上传模型到Vertex AI模型注册表

In [ ]:
serving_env = {
    "TASK": "text-to-image-lora",
    "MODEL_ID": "runwayml/stable-diffusion-v1-5",
    "FINETUNED_LORA_MODEL_PATH": os.path.join(
        MODEL_EXPORT_PATH, f"checkpoint_{deployed_checkpoint}"
    ),
    "DEPLOY_SOURCE": "notebook",
}

model = aiplatform.Model.upload(
    display_name=UPLOAD_MODEL_NAME,
    serving_container_image_uri=PREDICTION_CONTAINER_URI,
    serving_container_ports=[PREDICTION_PORT],
    serving_container_predict_route="/predictions/peft_serving",
    serving_container_health_route="/ping",
    serving_container_environment_variables=serving_env,
)

model.wait()

print("The uploaded model name is: ", UPLOAD_MODEL_NAME)

部署上传的模型

您将在Google Cloud Vertex AI中部署模型。默认设置将使用1个V100 GPU用于部署。

如果您尚未拥有服务帐户用于使用docker进行服务，请创建一个。

模型部署将需要大约1分钟才能完成。

In [ ]:
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
service_account = ""  # @param {type:"string"}

endpoint = aiplatform.Endpoint.create(display_name=f"{UPLOAD_MODEL_NAME}-endpoint")
model.deploy(
    endpoint=endpoint,
    machine_type=PREDICTION_MACHINE_TYPE,
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    accelerator_count=1,
    deploy_request_timeout=1800,
    service_account=service_account,
)

Docker容器在端点创建之后仍然需要下载并加载模型。因此，我们建议在继续下一个单元之前等待额外的3分钟。

部署完成后，您可以向端点发送一批文本提示以生成图像。

In [ ]:
import base64
from io import BytesIO

import matplotlib.pyplot as plt
from PIL import Image

instances = [
    {"prompt": "Two monadikos teapots on a table"},
    {"prompt": "Two monadikos teapots on the floor"},
]
response = endpoint.predict(instances=instances)

plt.figure()
_, grid = plt.subplots(1, len(instances))
for cell, prediction in zip(grid, response.predictions):
    image = Image.open(BytesIO(base64.b64decode(prediction)))
    cell.imshow(image)

将以下英文文本翻译为中文：## 清理工作
当导出完成后，您可以删除您的培训作业。

In [ ]:
if training_job.list(filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'):
    training_job.delete()
# Undeploys models and deletes endpoints.
endpoint.delete(force=True)
model.delete()

您也可以删除输出数据。

In [ ]:
!gsutil rm -r {STAGING_BUCKET}